In [1]:
# Obtains distance from rare variant to the TSS or TES
# distTSS - absolute distance to transcription start site
# distTES - absolute distance to transcription end site

In [2]:
import os

import numpy as np
import pandas as pd

In [3]:
def infostr_to_dict(infostr, kv_sep=' ', sep=';'):
    """Converts string to dictionary
    
    Parameters
    ----------
    infostr : str
        Info column from gencode
    
    kv_sep : str
        Separator between key and value
        
    sep : str
        Separator between key-value pairs
    """
    
    kv_pairs = infostr.rstrip(sep).split(sep)

    infodict = {}
    for pair in kv_pairs:
        #remove space in beginning
        pair = pair.lstrip()

        k, v = pair.split(kv_sep)
        v = v.replace('"','')
        infodict[k] = v
    
    return infodict

def infostr_to_gene_id(infostr):
    """Extracts gene_id from column 9 in gencode
    
    Paramters
    ---------
    infostr : str
        Info column from gencode
    """
    
    return infostr_to_dict(infostr)['gene_id']
    

def get_distance(rv_row):
    """Returns a rare variant's absolute distance to TSS and TES
    
    Parameters
    ----------
    rv_row : pandas.Series
        Row from rare variant file as dataframe
        
    Returns
    -------
    distTSS : int
        absolute distance to transcription start site
    distTSS : int
        absolute distance to transcription end site
    """
    
    pos = rv_row['Start'] + 1  # rare variant is 0-start, half-open, whereas gencode gtf is 1-start, fully-closed
    strand = rv_row['Strand']
    
    # positive strand
    if strand == '+':
        distTSS = np.abs(rv_row['genStart'] - pos)
        distTES = np.abs(rv_row['genEnd'] - pos)
    
    # negative strand
    else:
        distTSS = np.abs(rv_row['genEnd'] - pos)
        distTES = np.abs(rv_row['genStart'] - pos)        
    
    return distTSS, distTES

In [4]:
gencode_file = '/scratch/groups/abattle4/victor/WatershedAFR/raw_data/GTEx/gencode.v26.GRCh38.genes.gtf'
rv_file = '/scratch/groups/abattle4/victor/WatershedAFR/data/rare_variants_gnomad/gene-AFR-rv.txt'

In [5]:
# Read in gencode and rare variants
# gencode = pd.read_csv(gencode_file, names=['Chrom', 'Source', 'Feature', 'Start', 'End', 'Score','Strand', 'Phase', 'Info'], sep='\t', index_col=False, skiprows=6)
gencode = pd.read_csv(gencode_file, names=['Chrom', 'Source', 'Feature', 'genStart', 'genEnd', 'Score','Strand', 'Phase', 'Info'], sep='\t', index_col=False, skiprows=6)
rv = pd.read_csv(rv_file, sep='\t', index_col=False)

In [6]:
# Get gene_id from Info column
# gencode['gene_id'] = gencode['Info'].apply(infostr_to_gene_id)
gencode['Gene'] = gencode['Info'].apply(infostr_to_gene_id)

In [7]:
# Add strand, start, and end to rv
gencode_anno = pd.merge(rv, gencode[gencode['Feature'] == 'gene'][['Gene', 'genStart', 'genEnd', 'Strand']], how='left', on='Gene')

In [10]:
# compute distances from each RV to TSS and TES 
gencode_anno['distTSS'], gencode_anno['distTES'] = zip(*gencode_anno.apply(get_distance, axis=1))

In [11]:
# save gencode annotations
gencode_anno = gencode_anno[['Gene', 'Ind', 'Start', 'End', 'Ref', 'Alt', 'AF', 'distTSS', 'distTES']]
gencode_anno_file = '/scratch/groups/abattle4/victor/WatershedAFR/data/annotation/gene-AFR-rv.gencode.txt'
gencode_anno.to_csv(gencode_anno_file, sep='\t', index=False)